In [1]:
# %matplotlib notebook
import re
from optparse import OptionParser
from pyflann import *
import cv2
import numpy as np
import pandas as pd
import glob
import os
# import matplotlib.pyplot as plt

from bokeh.layouts import gridplot, column
from bokeh.plotting import figure, show, output_file
from bokeh.models import ColumnDataSource,LinearAxis, Range1d, Band
from bokeh.models.glyphs import Text
from bokeh.io import export_png, export_svgs

meta_data_header = ['FrameNumber', 'Time', 'Stage_x', 'Stage_y', 'Centroid_x', 'Centroid_y',
                    'Midpoint_x', 'Midpoint_y', 'Head_x', 'Head_y', 'Tail_x', 'Tail_y', 'MouthHook_x', 'MouthHook_y',
                    'LeftMHhook_x', 'LeftMHhook_y', 'RightMHhook_x', 'RightMHhook_y',
                    'LeftDorsalOrgan_x', 'LeftDorsalOrgan_y', 'RightDorsalOrgan_x', 'RightDorsalOrgan_y',
                    'CenterBolwigOrgan_x', 'CenterBolwigOrgan_y', 'LeftBolwigOrgan_x', 'LeftBolwigOrgan_y',
                    'RightBolwigOrgan_x', 'RightBolwigOrgan_y', 'V9_x', 'V9_y', 'V10_x', 'V10_y', 'V11_x', 'V11_y',
                    'V12_x', 'V12_y', 'V13_x', 'V13_y', 'V14_x', 'V14_y', 'V15_x', 'V15_y', 'V16_x', 'V16_y',
                    'MouthHook_votes', 'LeftMHhook_votes', 'RightMHhook_votes', 'LeftDorsalOrgan_votes',
                    'RightDorsalOrgan_votes', 'CenterBolwigOrgan_votes', 'LeftBolwigOrgan_votes', 'RightBolwigOrgan_votes',
                    'V9_votes', 'V10_votes', 'V11_votes', 'V12_votes', 'V13_votes', 'V14_votes', 'V15_votes',
                    'V16_votes', 'Num_Key_points']

coordinate_header = ['FrameNumber', 'MouthHook_x', 'MouthHook_y', 'LeftMHhook_x', 'LeftMHhook_y',
                     'RightMHhook_x', 'RightMHhook_y', 'LeftDorsalOrgan_x', 'LeftDorsalOrgan_y',
                     'RightDorsalOrgan_x', 'RightDorsalOrgan_y', 'CenterBolwigOrgan_x', 'CenterBolwigOrgan_y',
                     'LeftBolwigOrgan_x', 'LeftBolwigOrgan_y', 'RightBolwigOrgan_x', 'RightBolwigOrgan_y']


In [2]:
test_dir = '../expts/dataCollectedOn_20180417_withTestOn_20180619/dataCollectedOn_2018041_grp_2/20180529_150807/Rawdata_20180417_082627_20180525_112343_A_133410/'
video_path = '../expts/dataCollectedOn_20180417_withTestOn_20180619/videos/'

test_string, start_frame = ("Rawdata_20180417_082627", 149)

for fs in os.listdir(test_dir):
    if 'Metadata' in fs:
        meta_data = pd.read_csv(os.path.join(test_dir, fs), sep=',', header=0, names=meta_data_header)
    if 'Coordinates' in fs:
        coordinates = pd.read_csv(os.path.join(test_dir, fs), sep=',', names=coordinate_header)

if (meta_data.empty is False) and (coordinates.empty is False):
    meta_data.loc[:, 'FrameNumber'] = meta_data.loc[:, 'FrameNumber']
    meta_data.set_index('FrameNumber', inplace=True)
    
    coordinates = coordinates.astype('uint32')
    coordinates.loc[:, 'FrameNumber'] = coordinates.loc[:, 'FrameNumber'] - start_frame
    coordinates.set_index('FrameNumber', inplace=True)

#     plt.plot(meta_data.index.values,
#              meta_data['MouthHook_x'].values, 'b')
#     plt.plot(coordinates.index.values,
#          coordinates['MouthHook_x'].values, 'r')
#     plt.show()

In [3]:
p1 = figure(x_axis_type="linear", plot_width=1200, plot_height=300, toolbar_location='above')

# p1.background_fill_color = None
# p1.border_fill_color = None

p1.xaxis.axis_label = 'Time (sec)'
p1.yaxis.axis_label = 'Location'

# p1.line(meta_data.index.values, meta_data['MouthHook_x'].values, color='crimson', line_width=3)
p1.circle(meta_data.index.values, meta_data['MouthHook_x'].values, color="crimson", size=5)
# p1.line(coordinates.index.values, coordinates['MouthHook_x'].values, color='olive', line_width=3)
p1.circle(coordinates.index.values, coordinates['MouthHook_x'].values, color="blue", size=5)
# band = Band(base='time', lower='lower', upper='upper', source=source,
#             level='underlay', fill_color=trend_color,
#             fill_alpha=0.1, line_width=1, line_color=None)
# p1.add_layout(band)

# p1.xaxis.ticker = [0, 60, 120, 180, 240]
# p1.yaxis.ticker = [0, 1, 2]

#     # Setting the second y axis range name and range
#     p1.extra_y_ranges = {"Stimulus": Range1d(start=-3, end=52)}
#     p1.add_layout(LinearAxis(y_range_name="Stimulus", 
#                              axis_label="Stimulus W/m^2", 
#                              ticker=[0, 25, 50], axis_label_standoff=40), 'right')
#     p1.line(total_time, stimulus_watts, y_range_name='Stimulus', color='crimson')

# Dummy Glyph to add legend for the number of animals
# p1.circle(1, 1, color='white', legend='N = %d'%(num_animals))
# p1.yaxis.axis_label_standoff=30

# p1.xaxis.axis_label_text_font_size = "24pt"
# p1.yaxis.axis_label_text_font_size = "24pt"

# p1.xaxis.major_label_text_font_size = "18pt"
# p1.yaxis.major_label_text_font_size = "18pt"

# p1.xaxis.minor_tick_line_color = None  # turn off x-axis minor ticks
# p1.yaxis.minor_tick_line_color = None  # turn off y-axis minor ticks

# p1.xaxis.axis_line_width = 2
# p1.yaxis.axis_line_width = 2

# p1.xaxis.major_tick_line_width = 2
# p1.yaxis.major_tick_line_width = 2

# p1.legend.location = "top_right"
# p1.legend.glyph_width = 30
# p1.legend.label_text_font_size = "16pt"

# p1.xgrid.grid_line_color = None
# p1.ygrid.grid_line_color = None

# p1.xaxis.visible = None
# p1.yaxis.visible = None

# p1.outline_line_color = 'white'
# p1.legend.border_line_color = 'white'

show(p1)

In [4]:
p1 = figure(x_axis_type="linear", plot_width=1200, plot_height=300, toolbar_location='above')

p1.xaxis.axis_label = 'Time (sec)'
p1.yaxis.axis_label = 'Location'

p1.circle(meta_data.index.values, meta_data['MouthHook_y'].values, color="crimson", size=5)
p1.circle(coordinates.index.values, coordinates['MouthHook_y'].values, color="blue", size=5)

show(p1)

In [5]:
temp = pd.merge(meta_data, coordinates, on='FrameNumber',suffixes=('_T', '_A'))

In [6]:
a = temp.loc[:, ['MouthHook_x_T', 'MouthHook_x_A', 'MouthHook_y_T', 'MouthHook_y_A']]
a['Diff_X'] = a.loc[:, 'MouthHook_x_T'] - a.loc[:, 'MouthHook_x_A']
a['Diff_Y'] = a.loc[:, 'MouthHook_y_T'] - a.loc[:, 'MouthHook_y_A']
a

,MouthHook_x_T,MouthHook_x_A,MouthHook_y_T,MouthHook_y_A,Diff_X,Diff_Y
FrameNumber,,,,,,
2,0.0,1097,0.0,344,-1097.0,-344.0
3,1099.0,1095,363.0,364,4.0,-1.0
4,1098.0,1100,369.0,379,-2.0,-10.0
5,1093.0,1100,399.0,396,-7.0,3.0
6,1096.0,1098,409.0,419,-2.0,-10.0
16,1012.0,1017,458.0,480,-5.0,-22.0
18,977.0,980,392.0,397,-3.0,-5.0
19,977.0,981,364.0,366,-4.0,-2.0
20,983.0,983,339.0,341,0.0,-2.0
